In [6]:
import pickle
from sklearn.svm import SVC
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import time

In [7]:
df = pd.read_csv('preprocessed_data_PCA.csv',header=None)

In [41]:
# normalized_df=(df-df.mean())/df.std()
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.915478,-29.708285,-18.567470,19.941870,10.040662,47.994886,22.602111,-26.405726,23.781513,-5.840281,-2.202161,9.618834,-7.817354,-2.326412,0.830223
1,0.501408,-29.982723,-18.356527,20.196495,9.906214,48.728084,22.441971,-26.463517,23.666009,-6.104951,-2.466416,9.324974,-7.529465,-2.212196,1.018666
2,0.672313,-30.011755,-18.285465,19.496385,10.125876,48.823792,23.462359,-25.295018,23.042888,-6.194817,-2.579618,8.755442,-7.158226,-1.902847,1.055656
3,0.057232,-29.760668,-18.074979,20.012192,9.574128,48.935205,22.873641,-26.585023,23.323926,-6.535680,-2.766648,8.991219,-6.957730,-2.020536,1.184513
4,0.910670,-29.695378,-18.205501,19.553513,9.788785,49.051764,22.605893,-27.023506,23.661876,-6.404353,-2.656896,9.327718,-7.326092,-2.264918,0.889146


In [39]:
labels = pd.read_csv("labels.csv", header=None)

In [40]:
y = labels[1]

In [43]:
labels.columns = ['index', 'activityId']
print(labels.head())

   index  activityId
0   6147           1
1   6148           1
2   6149           1
3   6150           1
4   6151           1


In [44]:
joined_df = pd.concat([labels['activityId'], df], axis=1)

In [45]:
joined_df.head()

,activityId,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1,0.915478,-29.708285,-18.567470,19.941870,10.040662,47.994886,22.602111,-26.405726,23.781513,-5.840281,-2.202161,9.618834,-7.817354,-2.326412,0.830223
1,1,0.501408,-29.982723,-18.356527,20.196495,9.906214,48.728084,22.441971,-26.463517,23.666009,-6.104951,-2.466416,9.324974,-7.529465,-2.212196,1.018666
2,1,0.672313,-30.011755,-18.285465,19.496385,10.125876,48.823792,23.462359,-25.295018,23.042888,-6.194817,-2.579618,8.755442,-7.158226,-1.902847,1.055656
3,1,0.057232,-29.760668,-18.074979,20.012192,9.574128,48.935205,22.873641,-26.585023,23.323926,-6.535680,-2.766648,8.991219,-6.957730,-2.020536,1.184513
4,1,0.910670,-29.695378,-18.205501,19.553513,9.788785,49.051764,22.605893,-27.023506,23.661876,-6.404353,-2.656896,9.327718,-7.326092,-2.264918,0.889146


In [46]:
test_file = "ActivityDatasets/PCA/test_pca_activity_"
test_label = "ActivityDatasets/PCA/test_pca_label_"

for i in range(1, 25):
    test_file = "ActivityDatasets/test_pca_activity_"+str(i)
    test_label = "ActivityDatasets/test_pca_label_"+str(i)
    temp = joined_df[joined_df['activityId']==i][:200]
    temp[temp.columns[1:]].to_csv(test_file+".csv")
    temp['activityId'].to_csv(test_label+".csv")

In [1]:
# joined_df[joined_df['activityId']==joined_df['activityId'].max()]

In [20]:
# temp = joined_df[joined_df['activityId']==1][:20]

In [23]:
# temp[temp.columns[1:]].to_csv("test_activity.csv")

In [33]:
# temp['activityId']

In [17]:
# joined_df[joined_df.columns[1:]]

In [34]:
svd_df = pd.read_csv('preprocessed_data_SVD.csv',header=None)

In [35]:
# svd_normalized_df=(svd_df-svd_df.mean())/svd_df.std()

In [47]:
svd_joined_df = pd.concat([labels['activityId'], svd_df], axis=1)

In [48]:
svd_joined_df.head()

,activityId,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1,241.065407,4.568325,-29.462353,-16.344498,21.554520,8.727630,46.992144,-21.437557,-27.379202,24.916987,-6.506860,4.952574,-9.031316,3.972765,-0.521674
1,1,241.005587,4.138426,-29.741656,-16.198781,21.835562,8.575280,47.714613,-21.275756,-27.438019,24.804485,-6.779410,4.581905,-8.762475,3.773300,-0.523819
2,1,241.073656,4.292379,-29.770200,-16.070736,21.125088,8.825954,47.850472,-22.349457,-26.336326,24.138650,-6.850718,4.128201,-8.337516,3.462833,-0.352320
3,1,240.909646,3.684386,-29.524516,-15.926641,21.675685,8.243264,47.916351,-21.702392,-27.591276,24.469207,-7.219079,4.170502,-8.284885,3.365185,-0.576128
4,1,241.038717,4.556688,-29.451037,-15.879348,21.266759,8.466058,48.033721,-21.416052,-28.020814,24.825513,-7.094547,4.461557,-8.635757,3.677594,-0.646857


In [49]:
test_file = "ActivityDatasets/SVD/test_svd_activity_"
test_label = "ActivityDatasets/SVD/test_svd_label_"

for i in range(1, 25):
    test_file = "ActivityDatasets/SVD/test_svd_activity_"+str(i)
    test_label = "ActivityDatasets/SVD/test_svd_label_"+str(i)
    temp = svd_joined_df[svd_joined_df['activityId']==i][:200]
    temp[temp.columns[1:]].to_csv(test_file+".csv")
    temp['activityId'].to_csv(test_label+".csv")

In [53]:
svd_joined_df[svd_joined_df['activityId']==9][:2]

,activityId,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14


In [54]:
labels[labels['activityId'] == 9]

,index,activityId
